In [1]:
# import module list is independent on respective file.
import yfinance as yf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

from datetime import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,  MinMaxScaler
from scipy.stats import mstats
import japanize_matplotlib
from tqdm.notebook import tqdm

from sklearn.metrics import matthews_corrcoef

In [2]:
# load correct trading day format

csv_file_path = './data/Train_data_all.csv' # 後で書き込む
Train_data = pd.read_csv(csv_file_path, index_col=0, thousands=",", parse_dates=True) 
# read different lenghth for cutting
skip_to_date = "2005-11-01" 


# input the ticker
ticker = "IYR" #Real Estate SPDR  

#For IYR start from 2011/1, ends by 2023/4,

#     Load saved share price file
#file_path = '~/data/' #ダウンロードした人用
file_path = './data/' #自分用
file_name = ticker
file_extension = '.csv'
price_list = pd.read_csv(file_path + file_name + file_extension,thousands=",", index_col=0, parse_dates=True)           

In [3]:
# use correct format's index to retrieve the trading data
post_fundamental = Train_data[skip_to_date:]  
#https://qiita.com/NT1123/items/19a29e28a4184dbc84f5
idx = np.searchsorted(price_list.index, post_fundamental.index)
#documents = _list.iloc[idx]
fold_index= 3

In [4]:
def folding_train_data(df, index2):
    try:
        if "index" in df.columns.to_list():
            name = "index"
        else:
            df = df.reset_index(drop=False)
            name = "Date"
        df = df[df.index % index2==0]
        
        
        
    except Exception as e:
        raise e
    else:
        
        df.set_index(name,drop=True, inplace=True)
    
    return df 




In [5]:
fold_index= 3
#post_fundamental.rename_axis('Date', inplace=True)
post_fundamental = post_fundamental.loc[skip_to_date:, :].asfreq(str(fold_index)+'M')

In [32]:
post_fundamental

,Monthly Open,futures,Outlays - Receipts,US Bond,Outlays,Receipts,Government Account Series,State and Local Government Series,Treasury Inflation-Protected Securities,Principal Amount Balance,...,Net Unamortized Amount Increase,Principal Amount Decrease,Net Unamortized Amount Decrease,Accrued Interest Payable Amount,Accrued Interest Increase,Accrued Interest decrease,bond1,bond2,Floating_Rate,saving_securities
Date,,,,,,,,,,,,,,,,,,,,,
2005-11-30,2005/11/1,108.73,83072,4.500,221911,138840,3432818.00,231467,327907.0,4709403.0,...,-3420.0,329955.0,-3044.0,29717.0,16617.0,23149.0,5.166270e+05,196.6890,NaN,204578.7962
2006-02-28,2006/2/1,108.00,119237,4.547,232091,112853,3513085.00,234545,345551.0,4771117.0,...,-3981.0,302509.0,-2970.0,27100.0,11011.0,26855.0,5.267165e+05,218.1610,NaN,205900.7043
2006-05-31,2006/5/1,105.08,42907,5.113,235564,192657,3604230.00,239174,369651.0,4807517.0,...,-3583.0,397993.0,-3866.0,32222.0,16154.0,29508.0,5.231659e+05,200.4800,NaN,205669.4391
2006-08-31,2006/8/1,107.42,64717,4.732,218595,153878,3689211.00,238603,394485.0,4884685.0,...,-6172.0,416849.0,-3841.0,29637.0,15240.0,30604.0,5.347108e+05,237.9200,NaN,204024.7450
2006-11-30,2006/11/1,109.11,73042,4.458,218907,145866,3762962.00,247794,413418.0,4929567.0,...,-4871.0,425791.0,-4238.0,34033.0,12192.0,23902.0,5.306944e+05,255.1960,NaN,202800.0891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,2022/8/1,117.00,219596,3.133,523318,303722,6968348.00,99065,1824656.0,24267551.0,...,-12675.0,1488230.0,-5868.0,62610.0,54870.0,94269.0,3.844353e+06,6638.0155,603977.2267,164355.2943
2022-11-30,2022/11/1,113.50,248535,3.703,500646,252111,7157364.00,101819,1881707.0,24553457.0,...,-16499.0,1438035.0,-10401.0,68090.0,36213.0,62223.0,3.941889e+06,7011.6605,595214.1582,173217.6413
2023-02-28,2023/2/1,111.66,262434,3.916,524548,262114,6872104.00,99734,1876312.0,24609241.0,...,-17072.0,1568538.0,-13744.0,70658.0,26067.0,56827.0,4.033694e+06,7096.9704,579408.2858,177113.7872


In [33]:


DS = post_fundamental.loc[:,'Outlays - Receipts'].values

US_Bond = pd.read_csv('data/yieldsq.csv', index_col=0,parse_dates=True).loc['2005-11-30':'2023-09-30', 'Open'].values[::3]
Futures = post_fundamental.loc[:,'futures'].values

Receipts = post_fundamental.loc[:,'Receipts'].values

Outlays = post_fundamental.loc[:,'Outlays'].values

SLGS = post_fundamental.loc[:,'State and Local Government Series'].values

GAS = post_fundamental.loc[:,'Government Account Series'].values
TIPS = post_fundamental.loc[:,'Treasury Inflation-Protected Securities'].values

PAB = post_fundamental.loc[:,'Principal Amount  Balance'].values
AIPA = post_fundamental.loc[:,'Accrued Interest Payable Amount'].values

NUAB = post_fundamental.loc[:,'Net Unamortized Amount Balance'].values

Bill = post_fundamental.loc[:,'Bill(Net Sales Amount)'].values
Note = post_fundamental.loc[:,'Note(Net Sales Amount)'].values
Bond = post_fundamental.loc[:,'Bond(Net Sales Amount)'].values

SBI = post_fundamental.loc[:,'Saving Bond typeI'].values
SBEE = post_fundamental.loc[:,'Saving Bond typeEE'].values

PAI = post_fundamental.loc[:,'Principal Amount Increase'].values
AII = post_fundamental.loc[:,'Accrued Interest Increase'].values
NUAI = post_fundamental.loc[:,'Net Unamortized Amount Increase'].values
print('OK')
PAD = post_fundamental.loc[:,'Principal Amount Decrease'].values
AID = post_fundamental.loc[:,'Accrued Interest decrease'].values
NUAD = post_fundamental.loc[:,'Net Unamortized Amount Decrease'].values


bond1 = post_fundamental.loc[:,'bond1'].values
bond2 = post_fundamental.loc[:,'bond2'].values
SS = post_fundamental.loc[:,'saving_securities'].values

OK


In [8]:
file_name

'IYR'

In [34]:
# Goal is to anticipate the sign of future rate change from the financial data of the current monthly.
# If the future changes is + , we assign 1, otherwise 0,  to Future change value of the current quarter


# Returned Sales Amount.

# predict the FFO direction

data_2 = []
data_2 = pd.DataFrame(data_2)

# 2023 09 DS, Receipts, GAS, NUA
# 2023 09 DS Receipts Bond NUAB
data_2['DS'] = DS

data_2['SLGS'] = SLGS
data_2["GAS"] = GAS
data_2['Receipts'] = Receipts

# Principal Amount Balance
data_2['PAB'] = PAB
data_2['AIPA'] = AIPA
#data_2['Bill'] = Bill
#data_2['Note'] = Note
data_2['Bond'] = Bond


data_2['NUAB'] = NUAB
data_2['SBI'] = SBI
data_2['SBEE'] = SBEE
data_2['AII'] = AII
data_2['PAI'] = PAI
data_2['NUAI'] = NUAI

data_2['PAD'] = PAD 
data_2['AID'] = AID
data_2['NUAD'] = NUAD

data_2['bond1'] = bond1
data_2['bond2'] = bond2
data_2['SS'] = SS

In [45]:
bond=yf.download('^TNX', start='2001-01-01', end='2023-12-30', interval='1d')
array=pd.read_csv('data/datelist.csv', index_col=1, parse_dates=True).index
df = pd.DataFrame()
for i in array:
    df = pd.concat([df, bond.loc[i:, :].head(1)])
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-01-20,4.030,4.066,4.009,4.057,4.057,0
2004-02-17,4.036,4.059,4.019,4.048,4.048,0
2004-03-15,3.730,3.787,3.720,3.768,3.768,0
2004-04-19,4.324,4.374,4.312,4.372,4.372,0
2004-05-17,4.744,4.752,4.697,4.699,4.699,0
...,...,...,...,...,...,...
2023-05-15,3.502,3.511,3.466,3.508,3.508,0
2023-06-20,3.738,3.777,3.701,3.729,3.729,0
2023-07-17,3.779,3.836,3.779,3.797,3.797,0


In [15]:
post_fundamental.loc['2005-10-31']

,Monthly Open,futures,Outlays - Receipts,US Bond,Outlays,Receipts,Government Account Series,State and Local Government Series,Treasury Inflation-Protected Securities,Principal Amount Balance,...,Net Unamortized Amount Increase,Principal Amount Decrease,Net Unamortized Amount Decrease,Accrued Interest Payable Amount,Accrued Interest Increase,Accrued Interest decrease,bond1,bond2,Floating_Rate,saving_securities
Date,,,,,,,,,,,,,,,,,,,,,
2005-11-30,2005/11/1,108.73,83072,4.500,221911,138840,3432818.00,231467,327907.0,4709403.0,...,-3420.0,329955.0,-3044.0,29717.0,16617.0,23149.0,5.166270e+05,196.6890,NaN,204578.7962
2006-02-28,2006/2/1,108.00,119237,4.547,232091,112853,3513085.00,234545,345551.0,4771117.0,...,-3981.0,302509.0,-2970.0,27100.0,11011.0,26855.0,5.267165e+05,218.1610,NaN,205900.7043
2006-05-31,2006/5/1,105.08,42907,5.113,235564,192657,3604230.00,239174,369651.0,4807517.0,...,-3583.0,397993.0,-3866.0,32222.0,16154.0,29508.0,5.231659e+05,200.4800,NaN,205669.4391
2006-08-31,2006/8/1,107.42,64717,4.732,218595,153878,3689211.00,238603,394485.0,4884685.0,...,-6172.0,416849.0,-3841.0,29637.0,15240.0,30604.0,5.347108e+05,237.9200,NaN,204024.7450
2006-11-30,2006/11/1,109.11,73042,4.458,218907,145866,3762962.00,247794,413418.0,4929567.0,...,-4871.0,425791.0,-4238.0,34033.0,12192.0,23902.0,5.306944e+05,255.1960,NaN,202800.0891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,2022/8/1,117.00,219596,3.133,523318,303722,6968348.00,99065,1824656.0,24267551.0,...,-12675.0,1488230.0,-5868.0,62610.0,54870.0,94269.0,3.844353e+06,6638.0155,603977.2267,164355.2943
2022-11-30,2022/11/1,113.50,248535,3.703,500646,252111,7157364.00,101819,1881707.0,24553457.0,...,-16499.0,1438035.0,-10401.0,68090.0,36213.0,62223.0,3.941889e+06,7011.6605,595214.1582,173217.6413
2023-02-28,2023/2/1,111.66,262434,3.916,524548,262114,6872104.00,99734,1876312.0,24609241.0,...,-17072.0,1568538.0,-13744.0,70658.0,26067.0,56827.0,4.033694e+06,7096.9704,579408.2858,177113.7872


In [46]:
df.loc['2005-12-11':, :][::3]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-12-19,4.458,4.468,4.421,4.442,4.442,0
2006-03-20,4.674,4.679,4.638,4.656,4.656,0
2006-06-19,5.141,5.153,5.128,5.145,5.145,0
2006-09-18,4.802,4.848,4.802,4.810,4.810,0
2006-12-18,4.597,4.607,4.581,4.587,4.587,0
...,...,...,...,...,...,...
2022-09-19,3.510,3.510,3.463,3.490,3.490,0
2022-12-19,3.559,3.601,3.552,3.581,3.581,0
2023-03-20,3.395,3.507,3.395,3.481,3.481,0


In [47]:
US_Bond_change

array([[1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
 

In [48]:
US_Bond_change = np.where(df.loc['2005-12-11':, 'Open'][::3].shift(-1).diff()>0, 1, 0)
US_Bond_change = US_Bond_change[1:-1]
USBond_change = np.append(US_Bond_change,1) 
USBond_change

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1])

In [ ]:
''

In [15]:
df.to_csv('data/yieldsq.csv')

In [12]:
def model_learning(model, x, y, train_len=7):
    # reshapeする方法を考えればよい。
    for i in range(len(y)):
        try:
            model.fit(x[train_len-3+i:train_len-2+i],[y[train_len-2+i]])
        except Exception as e:
            print(e)
            pass
        else:
            print(x[train_len+i-3:train_len+i-2],y[train_len+i-2])
    return model

In [62]:

    print(col, int(num))

['AII','SLGS','SBI','SS'], 15
['AID','Bond','bond1','PAI'], 70
['SS','PAD','SBEE','NUAB'], 95
['SBEE','PAD','Receipts','SS'], 95
['AII','bond2','NUAB','AID'], 55
['NUAB','PAI','NUAD','bond2'], 45
['bond2','GAS','PAD','AIPA'], 35
['PAD','SBI','AIPA','bond2'], 15
['SBEE','Bond','SS','AID'], 45
['SLGS','SBEE','SS','PAB'], 40
['AID','SBI','SBEE','bond2'], 10
['bond1','GAS','SBEE','PAI'], 40
['PAI','bond1','NUAI','SBI'], 30
['NUAB','SLGS','bond1','SBI'], 15
['Bond','PAD','DS','bond2'], 10
['PAB','AII','NUAI','bond1'], 25
['PAD','bond1','NUAB','NUAI'], 60
['bond2','Bond','SBEE','GAS'], 95
['SBI','NUAD','SS','bond1'], 95
['AID','Bond','NUAB','bond2'], 25
['bond2','PAI','SS','SBEE'], 55
['SBI','bond1','NUAB','NUAD'], 55
['SS','SBI','AII','GAS'], 15
['AII','bond2','PAI','NUAD'], 5
['bond2','AIPA','PAD','PAI'], 10
['GAS','Bond','bond2','SLGS'], 50
['bond1','PAI','Receipts','bond2'], 95
['AIPA','SS','PAI','PAD'], 15
['NUAB','PAB','bond2','PAI'], 20
['NUAD','bond1','DS','NUAB'], 40
['bond1','PAD',

In [70]:
echo(col.rstrip(','))

(col.rstrip(','))


In [24]:
US_Bond

array([[4.45800018, 4.46799994, 4.421     , 4.44199991, 4.44199991,
        0.        ],
       [4.67399979, 4.6789999 , 4.63800001, 4.65600014, 4.65600014,
        0.        ],
       [5.14099979, 5.15299988, 5.12799978, 5.14499998, 5.14499998,
        0.        ],
       [4.80200005, 4.84800005, 4.80200005, 4.80999994, 4.80999994,
        0.        ],
       [4.59700012, 4.60699987, 4.58099985, 4.58699989, 4.58699989,
        0.        ],
       [4.56099987, 4.579     , 4.56099987, 4.5710001 , 4.5710001 ,
        0.        ],
       [5.13800001, 5.1880002 , 5.1329999 , 5.1420002 , 5.1420002 ,
        0.        ],
       [4.47399998, 4.49900007, 4.45800018, 4.46999979, 4.46999979,
        0.        ],
       [4.20699978, 4.23199987, 4.18200016, 4.19299984, 4.19299984,
        0.        ],
       [3.37299991, 3.41599989, 3.28800011, 3.31399989, 3.31399989,
        0.        ],
       [4.2249999 , 4.27299976, 4.20699978, 4.24499989, 4.24499989,
        0.        ],
       [3.52900004, 3

In [36]:
data_2

,DS,SLGS,GAS,Receipts,PAB,AIPA,Bond,NUAB,SBI,SBEE,AII,PAI,NUAI,PAD,AID,NUAD,bond1,bond2,SS
0,83072,231467,3432818.00,138840,4709403.0,29717.0,NaN,-36370.0,3.211405e+08,2761917.78,16617.0,388842.0,-3420.0,329955.0,23149.0,-3044.0,5.166270e+05,196.6890,204578.7962
1,119237,234545,3513085.00,112853,4771117.0,27100.0,NaN,-37870.0,1.543728e+08,2425929.68,11011.0,348989.0,-3981.0,302509.0,26855.0,-2970.0,5.267165e+05,218.1610,205900.7043
2,42907,239174,3604230.00,192657,4807517.0,32222.0,NaN,-37998.0,3.232138e+07,3107181.29,16154.0,385561.0,-3583.0,397993.0,29508.0,-3866.0,5.231659e+05,200.4800,205669.4391
3,64717,238603,3689211.00,153878,4884685.0,29637.0,NaN,-40549.0,1.121160e+07,2386459.61,15240.0,481802.0,-6172.0,416849.0,30604.0,-3841.0,5.347108e+05,237.9200,204024.7450
4,73042,247794,3762962.00,145866,4929567.0,34033.0,NaN,-40387.0,2.865996e+07,2301367.40,12192.0,474747.0,-4871.0,425791.0,23902.0,-4238.0,5.306944e+05,255.1960,202800.0891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,219596,99065,6968348.00,303722,24267551.0,62610.0,34763400.0,-63378.0,2.248403e+09,10255619.92,54870.0,1803202.0,-12675.0,1488230.0,94269.0,-5868.0,3.844353e+06,6638.0155,164355.2943
68,248535,101819,7157364.00,252111,24553457.0,68090.0,58259000.0,-88129.0,1.008583e+09,13452460.23,36213.0,1638950.0,-16499.0,1438035.0,62223.0,-10401.0,3.941889e+06,7011.6605,173217.6413
69,262434,99734,6872104.00,262114,24609241.0,70658.0,40378900.0,-92275.0,1.222408e+09,9736839.25,26067.0,1593785.0,-17072.0,1568538.0,56827.0,-13744.0,4.033694e+06,7096.9704,177113.7872
70,240348,96363,6835336.00,307487,24630686.0,77182.0,39781900.0,-99849.0,2.263807e+08,7941388.82,42979.0,1908861.0,-22265.0,1883243.0,89450.0,-16659.0,4.140504e+06,6858.1425,178582.1075


In [43]:
USBond_change

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,

In [50]:
len(y)

71

In [ ]:
#namelist = [['Receipts', 'AID', 'NUAI', 'AIPA']]
#namelist = [['PAD', 'NUAI', 'NUAD', 'NUAB']]

for num in range(5, 10, 5):
    col = ['AIPA', 'SS', 'PAI', 'PAD']

In [ ]:

Test1 ['AIPA', 'SS', 'PAI', 'PAD']
Test2 ['Receipts', 'AID', 'NUAI', 'AIPA']
Test3 ['AII', 'PAD', 'AID', 'NUAI']
Test4 ['NUAD', 'AII', 'PAD', 'NUAI']
Test5 ['PAB', 'NUAI', 'NUAD', 'AIPA']

In [70]:
#from xgboost import XGBClassifier  
#from lightgbm import LGBMClassifier, LGBMRegressor
#from catboost import CatBoostClassifier, CatBoostRegressor
#from sklearn.ensemble import RandomForestClassifier
#from lightgbm import LGBMClassifier
import warnings
warnings.simplefilter('ignore')

import ast

export=False 


train_len = 7
value2 = 0

  # adjustment, not being used
for num in range(5, 210, 5):
    col = ['PAB', 'NUAI', 'NUAD', 'AIPA']
    #num=40
    # 欠損値を列の1つ手前の値で埋める
    '''
    df = df.dropna(how='any').dropna(how='all', axis=0)
    df = df.fillna(method='ffill')
    '''

    print(col)
    X = data_2[col].values

    #USBond_change = np.where(US_Bond[1:] > US_Bond[0:-1], 1, 0)

    


    y = USBond_change


    y_test = []
    rate_prediction = []

    result = []
    result = pd.DataFrame(result)
    for i in range(len(y)-train_len+1):
        X_train = X[0:train_len+i-1,:]    # train until previous period, cuz y is forecasting next period
        y_train = y[0:train_len+i-1]
        X_predict = X[0:train_len+i,:] # use most recent reporting period for forecasting

        # Winsorize top 1% and bottom 1% of points.
        # Apply on X_train and X_test separately
        X_train = mstats.winsorize(X_train, limits = [0.01, 0.01])
        X_predict = mstats.winsorize(X_predict, limits = [0.01, 0.01])

        sc = StandardScaler()
        # Fit to training data and then transform it
        X_train = sc.fit_transform(X_train) # SVM is unsuitable to raw larger-scale number, However, Standard saves this.
        X_predict = sc.transform(X_predict) 

        y_test.append(y[train_len+i-1]) # 6, 7, 8, 9, 10, 11.......

        # Initialize svm, rbf is a default kernel
        classifier_rbf = SVC(C=200, kernel = 'rbf', gamma = 'auto', random_state=0)


        # Fit the model on training data

        # Make a prediction on testing data
        try:
            classifier_rbf.fit(X_train, y_train)
            y_pred_rbf = classifier_rbf.predict(X_predict[train_len -1+i,:].reshape(1,-1))
        except Exception as e:
            assert False, e
            pass


        rate_prediction = np.append(rate_prediction,y_pred_rbf)

    value= matthews_corrcoef(y_test, rate_prediction)

    

    result['predict'] = rate_prediction
    
    if value>value2:
        value2 = value
        print(num)
        print('corr:', value)
    else:
        
        continue 

    #print('corr:', value)







result.index = [i+timedelta(days=1) for i in post_fundamental.index[train_len:]]


            

['PAB', 'NUAI', 'NUAD', 'AIPA']
5
corr: 0.13933655548464177
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUAI', 'NUAD', 'AIPA']
['PAB', 'NUA

In [ ]:
corr: 0.15498260496951669

In [29]:
col

['AIPA', 'SS', 'PAI', 'PAD']

In [ ]:
90
corr: 0.15073529411764705
['AIPA', 'SS', 'PAI', 'PAD']

In [ ]:
corr: 0.21934449608111703
 corr: 0.15257352941176472   
    corr: 0.17955083174038333
         0.21586862835039825
    corr: 0.24803929333207597

In [ ]:
corr: 0.1840774878972323 Index(['AII', 'PAD', 'AID', 'NUAI'], dtype='object') 5


In [57]:
df = pd.DataFrame()
for i in a:
    dic = {}
    i = i.split(' ')
    dic['column'] =(''.join(i[2:-2]).replace('Index', '').replace('(', ''))
    dic['corr'], dic['C']  = [i[1]], [i[-1]]
    df = pd.concat([df, pd.DataFrame(dic)])
df

,column,corr,C
0,"['AII','SLGS','SBI','SS'],",0.21033353531577265,15\n
0,"['AID','Bond','bond1','PAI'],",0.22917462425705282,70\n
0,"['Receipts','NUAB','bond2','AIPA'],",0.21033353531577265,5\n
0,"['SS','PAD','SBEE','NUAB'],",0.14870813293635052,95\n
0,"['SBEE','PAD','Receipts','SS'],",0.21005023777259513,95\n
...,...,...,...
0,"['Receipts','bond2','SBI','PAI'],",0.2522522977486172,45\n
0,"['NUAI','PAB','PAI','bond2'],",0.15073529411764705,15\n
0,"['SBI','PAD','SS','PAI'],",0.24482305890331896,95\n
0,"['GAS','SS','NUAB','PAI'],",0.24803929333207597,45\n


In [58]:
df=df.reset_index(drop=True).drop_duplicates(subset=['corr'])

In [ ]:
 正解　：　〇['NUAD', 'AII', 'PAD', 'NUAI'] 
    11 -> ['AII', 'PAD', 'AID', 'NUAI']

(0.18039593813928767, ['NUAD', 'AII', 'PAD', 'NUAI'], 15)

In [ ]:


10 start C=20

['PAB', 'NUAI', 'NUAD', 'AIPA']

In [74]:
['NUAD', 'AII', 'PAD', 'NUAI'] 0.30z
['PAI', 'AII', 'NUAI', 'NUAD']
['Receipts', 'AID', 'NUAI', 'AIPA'],
['PAB', 'NUAI', 'NUAD', 'AIPA']]

In [ ]:
['DS', 'NUAD', 'NUAI', 'AIPA'

In [ ]:
rate_prediction

In [71]:
# Export data to CSV file
export = True
file_path = './prediction/'
file_name = "USBond"
file_extension = '3moMSQTest5.csv'

#     new_copy.to_csv(file_path + file_name + file_extension)
if export:
    data = result
    data.to_csv(file_path + file_name + file_extension)

In [ ]:
result.loc["2011-01-01":, :].sum()